<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Answering Medical Questions about Images using VantageCloud and Open-Source Language Models
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;'><b>Introduction:</b></p>

<p style='font-size:16px;font-family:Arial;'>In this comprehensive user demo, we will delve into the world of Medical Visual Question Answering using <b>Teradata Vantage</b> and <b>open-source language models</b>. This cutting-edge technology empowers businesses to uncover hidden insights from vast amounts of medical image data, enabling them to identify cells, diseases, artery etc.</p> 

<p style="font-size:16px;font-family:Arial;"><b>Key Features:</b></p>

<ol style="font-size:16px;font-family:Arial;">
<li><b>Improved Clinical Decision Support:</b> A well-trained medical VQA model enhances clinical decision-making by allowing healthcare providers to ask questions about medical images (e.g., X-rays, MRIs, CT scans) and receive accurate, rapid answers. This can lead to faster diagnoses and treatment plans.</li>

<li><b>Reducing Interpretation Errors:</b> Human interpretation of medical images can be subjective and prone to errors. A VQA model can provide objective, consistent, and evidence-based interpretations, helping to reduce diagnostic inaccuracies.</li>

<li><b>Time Efficiency:</b> The model's ability to quickly analyze images and answer questions can save valuable time for healthcare professionals, leading to more efficient patient care.</li>

<li><b>Accessibility:</b> Patients and non-specialist healthcare providers can benefit from a medical VQA system by obtaining easy-to-understand information about their health conditions, potentially improving health literacy.</li>

<li><b>Learning and Training Aid:</b> Medical VQA models can serve as educational tools for medical students, residents, and even experienced practitioners. They can be used to explain complex medical concepts and imaging findings.</li>

<li><b>Research Assistance:</b> Researchers can leverage the model to analyze large datasets of medical images more effectively. It can assist in extracting meaningful insights from these datasets, potentially leading to new discoveries in medical science.</li>

<li><b>Cross-Specialty Applicability:</b> A well-designed medical VQA model can be adapted to various medical specialties, from radiology and pathology to cardiology and dermatology. This versatility makes it a valuable asset across different healthcare domains.</li>

<li><b>Ethical Considerations:</b> It's essential to address ethical concerns related to privacy, security, and bias when deploying medical VQA models in healthcare settings. Ensuring patient data protection and model fairness is critical.</li>

<li><b>Continuous Improvement:</b> Model performance and accuracy should be continuously monitored and improved over time. Regular updates and retraining are necessary to keep up with evolving medical knowledge and technologies.</li>

<li><b>Collaboration:</b> Successful implementation of medical VQA models often requires collaboration between machine learning experts, healthcare professionals, and ethicists to ensure that the technology is used responsibly and effectively.</li>
</ol>


<p style = 'font-size:16px;font-family:Arial;'><b>Visual Question Answering:</b></p>
<p style = 'font-size:16px;font-family:Arial;'><b>What is visual Question Answering?</b></p>

<p style = 'font-size:16px;font-family:Arial;'>Visual Question Answering (VQA) is a task in computer vision that involves answering questions about an image. The goal of VQA is to teach machines to understand the content of an image and answer questions about it in natural language.</p>

<center><img id="125" src="./images/header_scene.png" height="400px" width="600px" style="border: 4px solid #404040; border-radius: 10px;"/></center>

<p style = 'font-size:16px;font-family:Arial;'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;'>
     <li>Configuring the environment</li>
  <li>Connect to Vantage</li>
  <li>Create a Custom Container in Vantage</li>
  <li>Install Dependencies</li>
  <li>Operationalizing AI-powered analytics</li>
  <li>Topic Modelling</li>
  <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial;'>1. Configuring the Environment</b>

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>1.1 Install the required libraries</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Please be aware that that it will take longer than 5 minutes to install these libraries the first time this cell is executed.</p>

In [ ]:
#%%capture
!pip install -r requirements.txt --quiet --no-warn-script-location
# !pip install --upgrade transformers --quiet --no-warn-script-location
# !pip install --upgrade torch --quiet --no-warn-script-location
# !pip install --upgrade sentence-transformers --quiet --no-warn-script-location

In [ ]:
%pip install -U python-dotenv teradataml huggingface-hub --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>Please restart the kernel after executing a </i><code>!pip install</code>. <i>The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i> and then clicking <b><i>Restart</i></b>.</p>

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>1.2 Import the required libraries</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Core imports
from teradataml import *
from teradatasqlalchemy.types import *

# Standard library imports
import os
import sys
import time
from time import sleep
import warnings
from collections import OrderedDict
from dotenv import load_dotenv, dotenv_values

# Utility imports
import shutil
from IPython.display import clear_output, display as ipydisplay

# Set display options for dataframes, plots, and warnings
warnings.filterwarnings('ignore')

%matplotlib inline

python_version = "3.11"
print(f'Using Python version {python_version} for user environment')

# Hugging Face model for the demo
model_names = {'ChetanHirapara/Salesforce-blip-vqa-base-medical-data'}

# a list of required packages to install in the custom OAF container
# modify this if using different models or design patterns
pkgs = ['transformers==4.57.6',
        'torch==2.10.0',
        'pandas==3.0.0',
        'sentence-transformers==5.2.0']

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>2. Connect to VantageCloud Lake</b></p>
<p style = 'font-size:16px;font-family:Arial'>Connect to VantageCloud using <code>create_context</code> from the teradataml Python library. If this environment has been prepared for connecting to a VantageCloud Lake OAF Container, all the details required will be loaded and you will see an acknowledgement after executing this cell.</p>

<p style = 'font-size:18px;font-family:Arial;'><b>2.1 Load the Environment Variables and Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Load the environment variables from a .env file and use them to create a connection context to Teradata.</p>

In [ ]:
print("Checking if this environment is ready to connect to VantageCloud Lake...")

if os.path.exists("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"):
    print("Your environment parameter file exist.  Please proceed with this use case.")
    # Load all the variables from the .env file into a dictionary
    env_vars = dotenv_values(
        "/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"
    )
    # Create the Context
    eng = create_context(
        host=env_vars.get("host"),
        username=env_vars.get("username"),
        password=env_vars.get("my_variable"),
    )
    execute_sql(
        """SET query_band='DEMO=Medical_Visual_Question_Answering.ipynb;' UPDATE FOR SESSION;"""
    )
    print("Connected to VantageCloud Lake with:", eng)
else:
    print("Your environment has not been prepared for connecting to VantageCloud Lake.")
    print("Please contact the support team.")

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial;'><b>2.2  Authenticate to the User Environment Service</b></p>

<p style = 'font-size:16px;font-family:Arial;'>To better support integration with Cloud Services and common automation tools; the <b > User Environment Service</b> is accessed via RESTful APIs.  These APIs can be called directly or in the examples shown below that leverage the Python Package for Teradata (teradataml) methods.</p> 

In [ ]:
# We've already loaded all the values into our environment variables and into a dictionary, env_vars.
# username=env_vars.get("username") isn't required when using base_url, pat and pem.

if set_auth_token(
    base_url=env_vars.get("ues_uri"),
    pat_token=env_vars.get("access_token"),
    pem_file=env_vars.get("pem_file"),
    valid_from=int(time.time()),
):
    print("UES Authentication successful")
else:
    print("UES Authentication failed. Check credentials.")
    sys.exit(1)

<hr style="height:2px;border:none;">

<b style = 'font-size:18px;font-family:Arial;'>3. Create a Custom Container in VantageCloud</b>

<p style = 'font-size:16px;font-family:Arial;'>If desired, the user can create a <b>new</b> custom environment by starting with a "base" image and customizing it.  The steps are:</p> 
<ul style = 'font-size:16px;font-family:Arial;'>
    <li>List the available "base" images the system supports</li>
    <li>List any existing "custom" environments the user has created</li>
    <li>If there are no custom environments, then create a new one from a base image</li>
    </ul>

In [ ]:
# Check if we have any existing environments
# If any other environments exist along with our default OAF environment, we will delete them
username = env_vars.get("username")
environment_name = username[: min(10, len(username))]

print(
    "Here is a list of the versions of the libraries available to be used within an OAF environments.\n"
)
print(list_base_envs())
env_list = list_user_envs()

if env_list is None:
    print("\nThis user does not have any environments.\nCreating your environment now.")
    demo_env = create_env(
        env_name=f"{environment_name}", base_env="python_3.11", desc="BYOLLM demo env"
    )
    print(demo_env)
else:
    print("\nHere is a list of your current environments:")
    ipydisplay(env_list)
    for env_name in env_list["env_name"]:
        if env_name == environment_name:
            demo_env = get_env(environment_name)
            print(
                "Your default environment already exists. You can continue with this notebook.\n\n"
            )
        else:
            print(
                f"Your existing environment, {env_name} doesn't match our default environment for this user."
            )
            print("We're going to delete it.")
            print(f"Please wait: Environment {env_name} is being removed!")
            remove_env(env_name)

<hr style='height:2px;border:none;'>

<p style = 'font-size:20px;font-family:Arial;'><b>4. Install Dependencies</b></p>

<p style = 'font-size:16px;font-family:Arial;'>The step in the process installs Python package dependencies. VantageCloud Open Analytics Framework Environments are secured against unauthorized access to the outside network. Users can load the required libraries and model using teradataml methods:
</p> 

<ul style = 'font-size:16px;font-family:Arial;'>
    <li>List the currently installed models and python libraries</li>
    <li><b>If necessary</b>, install any required packages</li>
    <li><b>If necessary</b>, install the pre-trained model.  This process takes several steps;
        <ol style = 'font-size:16px;font-family:Arial;'>
            <li>Import and download the model</li>
            <li>Create a zip archive of the model artifacts</li>
            <li>Call the install_model() method to load the model to the container</li>
        </ol></li>
    </ul>

In [ ]:
print("Here are the models installed in the remote user environment...")
ipydisplay(demo_env.models)

print("Here are the libraries installed in the remote user environment...")
ipydisplay(demo_env.libs)

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>4.1 A note on package versions</b></p>

<p style = 'font-size:16px;font-family:Arial;'>The next section of this demonstration makes use of the DataFrame apply() method, which will execute python code that we'll upload into the environment.  We have to ensue the python packages we have installed locally match the versions we have in the environment.
</p> 

<p style = 'font-size:16px;font-family:Arial;'><b>Note:</b> Please be aware that this step may take 15 minutes to execute.</p>

In [ ]:
# Get package versions using importlib instead of unsafe exec()
import importlib


def get_versions(pkg_names):
    """Get installed versions of packages without using exec()."""
    versioned_pkgs = []
    for pkg_name in pkg_names:
        try:
            pkg_module_name = pkg_name.replace("-", "_")
            module = importlib.import_module(pkg_module_name)
            version = getattr(module, "__version__", "unknown")
            versioned_pkgs.append(f"{pkg_name}=={version}")
        except ImportError:
            print(f"Warning: Package '{pkg_name}' not found")
    return versioned_pkgs


print(f"Checking required packages: {pkgs}")
v_pkgs = get_versions(pkgs)
installed_pkgs = set([x.split("==")[0] for x in pkgs])
environment_pkgs = set(demo_env.libs["name"].to_list())

if not installed_pkgs.issubset(environment_pkgs):
    missing_pkgs = installed_pkgs - environment_pkgs
    print(f"Installing missing packages: {missing_pkgs}")
    claim_id = demo_env.install_lib(
        [x.split("+")[0] for x in v_pkgs], asynchronous=True
    )
else:
    print(f"All required packages are installed in the {environment_name} environment")

In [ ]:
# claim_id = demo_env.install_lib(["transformers==4.57.6", "pandas==3.0.0", "torch==2.10.0", "sentence-transformers==5.2.0", "pillow==12.1.0"], asynchronous=True)

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>4.2 Monitor library installation status</b></p>

<p style = 'font-size:16px;font-family:Arial;'>Optionally - users can monitor the library installation status using the cell below:
</p> 

In [ ]:
# Check the status of installation using status() API.
# Create a loop here for demo purposes
try:
    claim_id
    ipydisplay(demo_env.status(claim_id))
    stage = demo_env.status(claim_id)["Stage"].iloc[-1]
    while stage == "Started":
        stage = demo_env.status(claim_id)["Stage"].iloc[-1]
        clear_output()
        ipydisplay(demo_env.status(claim_id))
        sleep(5)
except NameError:
    print("No installations to monitor")


# Verify the Python libraries have been installed correctly.
print(
    "Here is an updated list of libraries installed in the remote user environment..."
)
ipydisplay(demo_env.libs)

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>4.3 Download and install model</b></p>

<p style = 'font-size:16px;font-family:Arial;'>Open Analytics Framework environments do not have open access to the external network, which contributes to a very secure runtime environment.  As such, we will need to download the pre-trained model using the below API.  This cell will download the model and then install it into our OpenAnalytics Framework environment. This will take ~5 minutes.
</p> 

In [ ]:
from huggingface_hub import snapshot_download

MODEL_REPO_ID = "ChetanHirapara/Salesforce-blip-vqa-base-medical-data"
MODEL_DIR = "./blip_model"
MODEL_ZIP = "blip_model.zip"

try:
    print(f"Downloading model from {MODEL_REPO_ID}...")
    model_path = snapshot_download(repo_id=MODEL_REPO_ID, local_dir=MODEL_DIR)
    print(f"✓ Model downloaded to: {MODEL_DIR}")

    print("Creating archive for installation...")
    shutil.make_archive(
        MODEL_DIR,
        format="zip",
        root_dir=MODEL_DIR,
    )
    print(f"✓ Archive created: {MODEL_ZIP}")

    print("Installing model to environment...")
    claim_id = demo_env.install_model(MODEL_ZIP, asynchronous=True)
    print(f"✓ Model installation initiated with claim ID: {claim_id}")

except TeradataMlException as e:
    print(f"ℹ Model installation info: {e}")
except Exception as e:
    print(f"✗ Error during model setup: {e}")
    raise

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>4.4 Monitor model installation status</b></p>

<p style = 'font-size:16px;font-family:Arial;'>Optionally - users can monitor the model installation status using the cell below:
</p> 

In [ ]:
# Check the status of installation using status() API.
# Create a loop here for demo purposes
try:
    claim_id
    ipydisplay(demo_env.status(claim_id))
    stage = demo_env.status(claim_id)["Stage"].iloc[-1]
    while stage != "File Installed":
        stage = demo_env.status(claim_id)["Stage"].iloc[-1]
        clear_output()
        ipydisplay(demo_env.status(claim_id))
        sleep(5)
except NameError:
    print("No installations to monitor")


# Verify the model has been installed correctly.
demo_env.refresh()
ipydisplay(demo_env.models)

<p style = 'font-size:16px;font-family:Arial;'>The preceding demo showed how users can perform a <b>one-time</b> configuration task to prepare a custom environment for analytic processing at scale.  Once this configuration is complete, these containers can be re-used in ad-hoc development tasks, or used for operationalizing analytics in production.</p>

<hr style='height:2px;border:none;'>
<p style = 'font-size:20px;font-family:Arial;'><b>5. Operationalizing AI-powered analytics</b></p>
<p style = 'font-size:16px;font-family:Arial;'>The following demonstration will illustrate how developers can take the next step in the process to <b>operationalize</b> this processing, enabling the entire organization to leverage AI across the data lifecycle, including</p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '30%'>
           <ol style = 'font-size:16px;font-family:Arial;'>
               <li><b>Prepare the environment</b>.  Package the scoring function into a more robust program, and stage it on the remote environment</li>
            <br>
            <br>
               <li><b>Python Pipeline</b>.  Execute the function using Python methods</li>
            <br>
            <br>
               <li><b>SQL Pipeline</b>.  Execute the function using SQL - allowing for broad adoption and use in ETL and operational needs</li>
        </ol>
        </td>
        <td width = '20%'></td>
        <td style = 'vertical-align:top'><img src = 'images/OAF_Ops.png' width=350 style="border: 4px solid #404040; border-radius: 10px;"/></td>
    </tr>
</table>


<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial;'><b>5.1 Create a server-side Visual inference function</b></p>

<p style = 'font-size:16px;font-family:Arial;'>The goal of this exercise is to create a <b>server-side</b> function which can be staged on the analytic cluster.  This offers many improvements over the method used above;</p> 
<ul style = 'font-size:16px;font-family:Arial;'>
    <li><b>Performance</b>.  Staging the code and dependencies in the container environment reduces the amount of I/O, since the function doesn't need to get serialized to the cluster when called</li>
    <li><b>Operationalization</b>.  The execution pipeline can be encapsulated into a SQL statement, which allows for seamless use in ETL pipelines, dashboards, or applications that need access</li>
    <li><b>Flexibility</b>. Developers can express much greater flexibility in how the code works to optimize for performance, stability, data cleanliness or flow logic</li>
</ul>

<p style = 'font-size:16px;font-family:Arial;'>These benefits do come with some amount of additional work.  Developers need to accomodate how data is passed in and out of the code at runtime, and how to pass it back to the SQL engine to assemble and return the final result set.  Code is executed when the user submits an <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/SQL-Reference/SQL-Operators-and-User-Defined-Functions/Table-Operators/APPLY'>APPLY SQL function</a>;</p> 
<ol style = 'font-size:16px;font-family:Arial;'>
    <li><b>Input Query</b>.  The <code>APPLY</code> function takes a SQL query as input.  This query can be as complex as needed and include data preparation, cleansing, and/or any other set-based logic necessary to create the desired input data set.  This complexity can also be abstracted into a database view.  When using the teradata client connectors for Python or R, thise query is represented as a DataFrame or tibble.</li>
    <li><b>Pre-processing</b>.  Based on the query plan, data is retrieved from storage (cache, block storage, or object storage) and the input query is executed.</li>
    <li><b>Distribution</b>.  Input data can be partitioned and/or ordered to be processed on a specific container or collection of them.  For example, the user may want to process all data for a single post code in one partition, and run thousands of these in parallel.  Data can also be distributed evenly across all units of parallelism in the system</li>
    <li><b>Input</b>.  The data for each container is passed to the runtime using tandard input (stdin)</li>
    <li><b>Processing</b>.  The user's code executes, parsing stdin for the input data</li>
    <li><b>Output</b>.  Data is sent out of the code block using standard output (stdout)</li>
    <li><b>Resultset</b>.  Resultset is assembled by the analytic database, and the SQL query returns</li>
    </ol>


# Expose a helper to read the current selection later
def get_selected_path():
    """Return the single selected file path or None."""
    try:
        selected = next((p for p, cb in zip(image_paths, selectors) if cb.value), None)
        return selected
    except (NameError, IndexError):
        return None

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>5.3.  Install the file and any additional artifacts</b></p>

<p style = 'font-size:16px;font-family:Arial;'>Use the install_file() method to install this python file to the container.  As a reminder, this container is persistent, so these steps need only be done infrequently.</p> 

In [ ]:
import os

folder_name = "library"

# Check if the folder exists
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created.")
else:
    print(f"Folder '{folder_name}' already exists.")

In [ ]:
%%writefile ./library/Medical_Visual_Question_Answering_OAF.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Medical Visual Question Answering using BLIP model.

This module provides functionality for performing visual question answering
on medical images using the BLIP (Bootstrapped Language-Image Pre-training) model.
It reads input data from stdin, processes images and questions, and outputs predictions.

Created on Mon Sept 29 18:54:17 2025
@author: author
"""
import sys
import os
import warnings
import pandas as pd
import base64
from io import BytesIO
from pathlib import Path
import time
import getpass
from typing import Tuple, Dict, List, Any
import torch
from PIL import Image

warnings.simplefilter("ignore")

# ============================================================================
# CONFIGURATION CONSTANTS
# ============================================================================
# Model configuration
MODEL_PATH = "./models/blip_model"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Image processing parameters
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_SIZE = 224
DEFAULT_IMAGE_COLOR = (200, 100, 50)

# Text processing parameters
MAX_TEXT_LENGTH = 32
MAX_GENERATION_LENGTH = 50
NUM_BEAMS = 3

# Data processing
INPUT_DELIMITER = "#"
REQUIRED_COLUMNS = ["id", "img", "question", "answer"]
REQUIRED_INPUT_COLUMNS = ["question", "answer", "img"]

# Cache directories
CACHE_DIRS = {
    "torch": "/tmp/torch_cache",
    "huggingface": "/tmp/huggingface_cache",
    "transformers": "/tmp/transformers_cache",
    "torch_inductor": "/tmp/torch_inductor_cache",
}


original_getuser = getpass.getuser


def safe_getuser() -> str:
    """
    Safe wrapper for getuser that handles UID lookup failures in containerized envs.

    Returns:
    --------
    str : Username or fallback UID string
    """
    try:
        return original_getuser()
    except KeyError:
        return f"uid_{os.getuid()}"


# Monkey-patch getpass before torch imports it
getpass.getuser = safe_getuser

# Configure cache directories before importing torch/transformers
for cache_name, cache_path in CACHE_DIRS.items():
    os.environ[
        (
            f"TORCH_HOME"
            if cache_name == "torch"
            else (
                f"HF_HOME"
                if cache_name == "huggingface"
                else (
                    f"TRANSFORMERS_CACHE"
                    if cache_name == "transformers"
                    else "TORCHINDUCTOR_CACHE_DIR"
                )
            )
        )
    ] = cache_path

    try:
        os.makedirs(cache_path, exist_ok=True)
    except OSError:
        pass

from transformers import (
    BlipProcessor,
    BlipForQuestionAnswering,
    BlipImageProcessor,
    BlipConfig,
)
import numpy as np


def load_input_data() -> pd.DataFrame:
    """
    Load and parse input data from stdin.

    Expected format: "id#image_base64#question#answer" (one per line)

    Returns:
    --------
    pd.DataFrame : DataFrame with columns: id, img, question, answer

    Raises:
    -------
    ValueError : If no input data provided
    RuntimeError : If data format is invalid
    """
    try:
        inputData = []

        for line_num, line in enumerate(sys.stdin.read().splitlines(), 1):
            try:
                parts = line.split(INPUT_DELIMITER)
                if len(parts) < 4:
                    raise ValueError(f'Expected 4 fields, got {len(parts)}"')
                inputData.append(parts)
            except ValueError:
                continue

        if not inputData:
            raise ValueError("No input data provided")

        pdf = pd.DataFrame(inputData, columns=REQUIRED_COLUMNS).copy()
        return pdf
    except Exception as e:
        raise


def initialize_model() -> (
    Tuple[BlipForQuestionAnswering, BlipProcessor, BlipImageProcessor, torch.device]
):
    """
    Initialize BLIP model components.

    Returns:
    --------
    tuple : (model, text_processor, image_processor, device)
        - model: BlipForQuestionAnswering instance
        - text_processor: BlipProcessor for text
        - image_processor: BlipImageProcessor for images
        - device: torch.device (cuda or cpu)

    Raises:
    -------
    FileNotFoundError : If model files not found at MODEL_PATH
    RuntimeError : If model loading fails
    """
    try:
        model_path = Path(MODEL_PATH)
        if not model_path.exists():
            raise FileNotFoundError(f"Model path not found: {MODEL_PATH}")

        # Load model components
        try:
            config = BlipConfig.from_pretrained(MODEL_PATH)
        except Exception as e:
            raise RuntimeError(f"Failed to load model config: {e}") from e

        try:
            text_processor = BlipProcessor.from_pretrained(MODEL_PATH)
        except Exception as e:
            raise RuntimeError(f"Failed to load text processor: {e}") from e

        try:
            image_processor = BlipImageProcessor.from_pretrained(MODEL_PATH)
        except Exception as e:
            raise RuntimeError(f"Failed to load image processor: {e}") from e

        try:
            model = BlipForQuestionAnswering.from_pretrained(MODEL_PATH)
        except Exception as e:
            raise RuntimeError(f"Failed to load model: {e}") from e

        model.to(DEVICE)
        return model, text_processor, image_processor, DEVICE

    except (FileNotFoundError, RuntimeError):
        raise
    except Exception as e:
        raise RuntimeError(f"Model initialization failed: {e}") from e


class VQADataset(torch.utils.data.Dataset):
    """
    Visual Question Answering Dataset for BLIP model.

    Handles image decoding from base64, text processing, and label encoding.
    """

    def __init__(
        self,
        data: pd.DataFrame,
        text_processor: BlipProcessor,
        image_processor: BlipImageProcessor,
    ):
        """
        Initialize VQA dataset.

        Parameters:
        -----------
        data : pd.DataFrame
            DataFrame with columns: question, answer, img
        text_processor : BlipProcessor
            Text processing pipeline
        image_processor : BlipImageProcessor
            Image processing pipeline
        """
        self.data = data
        self.text_processor = text_processor
        self.image_processor = image_processor
        self.max_length = MAX_TEXT_LENGTH
        self.image_height = IMAGE_HEIGHT
        self.image_width = IMAGE_WIDTH

        if hasattr(data, "iloc"):
            self.questions = data["question"].tolist()
            self.answers = data["answer"].tolist()
            self.images = data["img"].tolist()
        else:
            self.questions = data["question"]
            self.answers = data["answer"]
            self.images = data["img"]

    def __len__(self) -> int:
        """Return dataset size."""
        return len(self.questions)

    def _decode_image(self, img_data: str) -> Image.Image:
        """
        Decode image from base64 or binary data.

        Parameters:
        -----------
        img_data : str or bytes
            Base64-encoded or binary image data

        Returns:
        --------
        Image.Image : PIL Image object in RGB format
        """
        try:
            if isinstance(img_data, str):
                img_bytes = base64.b64decode(img_data)
            else:
                img_bytes = img_data

            image = Image.open(BytesIO(img_bytes)).convert("RGB")
            return image
        except (ValueError, OSError):
            return Image.new("RGB", (224, 224), color="white")
        except Exception:
            return Image.new("RGB", (224, 224), color="white")

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        """
        Get a single sample.

        Parameters:
        -----------
        idx : int
            Sample index

        Returns:
        --------
        dict : Encoded image and text with labels
        """
        try:
            answers = self.answers[idx]
            questions = self.questions[idx]
            image = self._decode_image(self.images[idx])
            text = self.questions[idx]

            image_encoding = self.image_processor(
                image,
                do_resize=True,
                size=(self.image_height, self.image_width),
                return_tensors="pt",
            )

            encoding = self.text_processor(
                None,
                text,
                padding="max_length",
                truncation=True,
                max_length=self.max_length,
                return_tensors="pt",
            )

            for k, v in encoding.items():
                encoding[k] = v.squeeze()
            encoding["pixel_values"] = image_encoding["pixel_values"][0]

            labels = self.text_processor.tokenizer.encode(
                answers,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            )[0]
            encoding["labels"] = labels

            return encoding
        except Exception:
            raise


def inference_single(
    input_df: pd.DataFrame,
    model: BlipForQuestionAnswering,
    text_processor: BlipProcessor,
    image_processor: BlipImageProcessor,
    device: torch.device,
    sample_idx: int = 0,
    visualize: bool = False,
) -> Dict[str, Any]:
    """
    Perform Visual Question Answering inference on a single sample.

    Parameters:
    -----------
    input_df : pd.DataFrame
        DataFrame containing columns: 'question', 'answer', 'img'
    model : BlipForQuestionAnswering
        Pre-loaded BLIP model
    text_processor : BlipProcessor
        Text processor for encoding/decoding
    image_processor : BlipImageProcessor
        Image processor for preprocessing
    device : torch.device
        Device for inference (cuda or cpu)
    sample_idx : int, default=0
        Index of the sample to process
    visualize : bool, default=False
        Whether to display the input image

    Returns:
    --------
    dict : Inference results with keys:
        - question: Input question text
        - predicted_answer: Model-generated answer
        - actual_answer: Ground truth answer
        - sample_index: Index of processed sample
        - processing_time: Total processing time in seconds
        - inference_time: Model inference time in seconds
        - device_used: Device used for inference

    Raises:
    -------
    ValueError : If input validation fails
    RuntimeError : If inference fails
    """
    start_time = time.time()

    try:
        # Validate input
        missing_columns = [
            col for col in REQUIRED_INPUT_COLUMNS if col not in input_df.columns
        ]
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")

        if len(input_df) == 0:
            raise ValueError("Input DataFrame is empty")

        if sample_idx >= len(input_df):
            raise ValueError(
                f"Sample index {sample_idx} out of range [0, {len(input_df)-1}]"
            )

        # Create dataset and get sample
        val_vqa_dataset = VQADataset(
            data=input_df,
            text_processor=text_processor,
            image_processor=image_processor,
        )

        sample = val_vqa_dataset[sample_idx]

        question_text = text_processor.decode(
            sample["input_ids"], skip_special_tokens=True
        )
        actual_answer = text_processor.decode(
            sample["labels"], skip_special_tokens=True
        )

        # Prepare batch
        sample_batch = {k: v.unsqueeze(0).to(device) for k, v in sample.items()}

        # Run inference
        model.eval()
        inference_start = time.time()

        try:
            with torch.no_grad():
                outputs = model.generate(
                    pixel_values=sample_batch["pixel_values"],
                    input_ids=sample_batch["input_ids"],
                    max_length=MAX_GENERATION_LENGTH,
                    num_beams=NUM_BEAMS,
                    early_stopping=True,
                    do_sample=False,
                )
        except RuntimeError as e:
            raise RuntimeError(f"Model generation failed: {e}") from e

        inference_time = time.time() - inference_start
        predicted_answer = text_processor.decode(outputs[0], skip_special_tokens=True)

        results = {
            "question": question_text,
            "predicted_answer": predicted_answer,
            "actual_answer": actual_answer,
            "sample_index": sample_idx,
            "processing_time": time.time() - start_time,
            "inference_time": inference_time,
            "device_used": str(device),
        }

        return results

    except (ValueError, RuntimeError):
        raise
    except Exception as e:
        raise RuntimeError(f"Inference failed: {e}") from e


def process_batch_inference(
    pdf: pd.DataFrame,
    model: BlipForQuestionAnswering,
    text_processor: BlipProcessor,
    image_processor: BlipImageProcessor,
    device: torch.device,
) -> List[Dict[str, str]]:
    """
    Process all rows in the DataFrame and return results.

    Parameters:
    -----------
    pdf : pd.DataFrame
        Input DataFrame with columns: id, img, question, answer
    model : BlipForQuestionAnswering
        Pre-loaded model
    text_processor : BlipProcessor
        Text processor
    image_processor : BlipImageProcessor
        Image processor
    device : torch.device
        Inference device

    Returns:
    --------
    list : List of result dictionaries with keys: id, question, answer, predicted_answer

    Raises:
    -------
    RuntimeError : If batch processing fails critically
    """
    results = []

    try:
        for index, row in pdf.iterrows():
            try:
                single_row_df = pd.DataFrame([row])
                result = inference_single(
                    single_row_df,
                    model,
                    text_processor,
                    image_processor,
                    device,
                    sample_idx=0,
                    visualize=False,
                )
                results.append(
                    {
                        "id": row["id"],
                        "question": result["question"],
                        "answer": result["actual_answer"],
                        "predicted_answer": result["predicted_answer"],
                    }
                )
            except Exception:
                # Add error result to maintain row count
                results.append(
                    {
                        "id": row["id"],
                        "question": row["question"],
                        "answer": row["answer"],
                        "predicted_answer": "ERROR",
                    }
                )

        return results

    except Exception as e:
        raise RuntimeError(f"Batch processing failed: {e}") from e


# ============================================================================
# MAIN EXECUTION
# ============================================================================


def main():
    """Main entry point for Visual Question Answering inference."""
    try:
        # Load input data
        pdf = load_input_data()

        # Initialize model
        model, text_processor, image_processor, device = initialize_model()

        # Process batch
        results = process_batch_inference(
            pdf, model, text_processor, image_processor, device
        )

        # Output results
        for result in results:
            print(
                result["id"],
                INPUT_DELIMITER,
                result["question"],
                INPUT_DELIMITER,
                result["answer"],
                INPUT_DELIMITER,
                result["predicted_answer"],
            )

    except ValueError:
        sys.exit(1)
    except (FileNotFoundError, RuntimeError):
        sys.exit(2)
    except Exception:
        sys.exit(3)


if __name__ == "__main__":
    main()

In [ ]:
file_path = "./library/Medical_Visual_Question_Answering_OAF.py"
demo_env.install_file(file_path=file_path, replace=True)

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>5.5  Call the APPLY function </b></p>
<p style = 'font-size:16px;font-family:Arial;'>This function can be executed in two ways;</p> 
<ul style = 'font-size:16px;font-family:Arial;'>
    <li><b><a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Teradata-Package-for-Python-on-VantageCloud-Lake/Working-with-Open-Analytics/teradataml-Apply-Class-for-APPLY-Table-Operator'>Python</a></b> by calling the Apply() module function</li>
    <li><b><a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/SQL-Reference/SQL-Operators-and-User-Defined-Functions/Table-Operators/APPLY'>SQL</a></b> which allows for broad adoption across the enterprise</li>
    </ul>
    

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>5.6 APPLY using Python</b></p>

<p style = 'font-size:16px;font-family:Arial;'>The process is as follows</p> 
<ol style = 'font-size:16px;font-family:Arial;'>
    <li>Construct a dictionary that will define the return columns and data types</li>
    <li>Construct a teradataml DataFrame representing the data to be processed - note this is a "virtual" object representing data and logic <b>in-database</b></li>
    <li>Execute the module function.  This constructs the function call in the database, but does not execute anything.  Note the Apply function takes several arguments - the input data, environment name, and the command to run</li>
    <li>In order to execute the function, an "execute_script()" method must be called.  This method returns the server-side DataFrame representing the complete operation.  This DataFrame can be used in further processing, stored as a table, etc.</li>
    </ol>
    

In [ ]:
tdf = DataFrame.from_query("""select * from DEMO_RefData.Medical_Images""")

In [ ]:
tdf

In [ ]:
def inference(tdf):
    # return types dictionary
    types_dict = OrderedDict({})
    types_dict["id"] = VARCHAR(100)
    types_dict["question"] = VARCHAR(1000)
    types_dict["answer"] = VARCHAR(1000)
    types_dict["predicted_answer"] = VARCHAR(1000)

    apply_obj = Apply(
        data=tdf,
        apply_command=f"python Medical_Visual_Question_Answering_OAF.py",
        returns=types_dict,
        env_name=demo_env,
        delimiter="#",
        quotechar="@",
    )

    return apply_obj.execute_script()

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>5.7 Execute the function</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Call <code>execute_script()</code>, and return a single record to the client to check the data.</p> 

In [ ]:
## GPU_Compute_Group = env_vars.get("gpu_compute_group")
if not GPU_Compute_Group:
    print("Error: gpu_compute_group is not set!")
else:
    print("Setting Compute Group for this session.")
    execute_sql(f"SET SESSION COMPUTE GROUP {GPU_Compute_Group};")
    print("\nNow executing the inference...")
    visual_question_df = inference(tdf)
    ipydisplay(visual_question_df)

<hr style='height:1px;border:none;'>
<p style = 'font-size:16px;font-family:Arial;'>If something goes wrong during the <code>Apply()</code> you will be informed that you can execute the <code>view_log()</code> function to download the logs.  Uncomment this cell and paste in the id you will be presented with.</p> 

In [ ]:
visual_question_pdf = visual_question_df.to_pandas()
visual_question_pdf["id"] = visual_question_pdf["id"].astype(int)

In [ ]:
visual_question_pdf.sort_values("id")

<p style = 'font-size:16px;font-family:Arial;'>Now the results can be saved back to Vantage.</p> 

In [ ]:
copy_to_sql(
    df=visual_question_df, table_name="visual_question_prediction", if_exists="replace"
)

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>5.8 Chatbot</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Now we can build the interactive chatbot to ask questions. You can only submit one file at a time -- click on the checkbox to select an image.</p> 

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from PIL import Image as PILImage
import time
import io
import os
import glob
import random
import threading
import html as html_module

# ---------- Configuration ----------
IMAGE_DIR = "./medical_images"  # Folder to scan
THUMB_SIZE = (180, 180)  # Max width/height for thumbnails in the grid

# Dictionary with counter values as keys
data_dict = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

# Initialize counter
counter = 0

# Custom CSS for modern styling
custom_css = """
<style>
    .main-container {
        background: #00233c;
        padding: 40px;
        border-radius: 20px;
        box-shadow: 0 15px 35px rgba(0,0,0,0.4);
        max-width: 900px;
        margin: 30px auto;
    }
    .app-title {
        color: #fc5f21;
        font-size: 32px;
        font-weight: bold;
        text-align: center;
        margin-bottom: 0px;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        background: #00233c;
    }
    .app-subtitle {
        color: #F0F0F0;
        text-align: center;
        margin-bottom: 20px;
        font-size: 15px;
        #opacity: 0.8;
        background: #00233c;
    }
    .section-block {
        background: rgba(0, 35, 60, 0.8);
        padding: 25px;
        border-radius: 12px;
        margin: 20px 0;
        border: 2px solid rgba(255, 86, 3, 0.8);
        backdrop-filter: blur(10px);
    }
    .section-title {
        color: #fc5f21;
        font-size: 18px;
        font-weight: bold;
        margin-bottom: 15px;
        padding-bottom: 10px;
        border-bottom: 2px solid rgba(255, 140, 0, 0.3);
    }
</style>
"""


# ---------- Helpers ----------
def list_pngs(dirpath):
    """List all PNG files in a directory."""
    os.makedirs(dirpath, exist_ok=True)
    return sorted(glob.glob(os.path.join(dirpath, "*.png")))


def make_thumb_bytes(path, max_wh=THUMB_SIZE):
    """Read an image, generate a PNG thumbnail, and return bytes."""
    with PILImage.open(path) as im:
        # Preserve transparency if any, otherwise use RGB
        if im.mode not in ("RGB", "RGBA"):
            im = im.convert("RGBA")
        im.thumbnail(max_wh, PILImage.LANCZOS)
        bio = io.BytesIO()
        im.save(bio, format="PNG")
        return bio.getvalue()


# Section 1: Input Section
# Scan images & build UI pieces
image_paths = list_pngs(IMAGE_DIR)

if not image_paths:
    gallery = widgets.VBox(
        [
            widgets.HTML(
                '<div class="section-title">📥 Select one image from ./medical_images</div>'
            ),
            widgets.HTML(
                '<p style="color:#F0F0F0;opacity:.8;margin:0">No PNGs in <code>./medical_images</code>.</p>'
            ),
        ],
        layout=widgets.Layout(
            padding="25px",
            background="rgba(0, 35, 60, 0.8)",
            border="2px solid rgba(255, 86, 3, 0.8)",
            border_radius="12px",
            margin="20px 0",
        ),
    )
else:
    cards = []
    selectors = []
    state = {"updating": False}  # <-- avoids nonlocal/global

    # Large preview
    preview = widgets.Image(
        format="png",
        layout=widgets.Layout(
            width="400px",
            height="auto",
            border="1px solid rgba(255,86,3,0.3)",
            padding="6px",
            border_radius="8px",
            background="rgba(0,35,60,0.5)",
        ),
    )

    # Build each card (thumbnail + checkbox acting like radio)
    for idx, p in enumerate(image_paths):
        thumb = make_thumb_bytes(p)
        img = widgets.Image(
            value=thumb,
            format="png",
            layout=widgets.Layout(width="140px", height="auto"),
        )
        cb = widgets.Checkbox(
            value=False,
            description=os.path.basename(p),
            indent=False,
            layout=widgets.Layout(width="160px"),
        )
        selectors.append(cb)

        card = widgets.VBox(
            [img, cb],
            layout=widgets.Layout(
                align_items="center",
                border="1px solid rgba(255,86,3,0.25)",
                padding="8px",
                border_radius="8px",
                background="rgba(0, 35, 60, 0.5)",
            ),
        )
        cards.append(card)

        def on_change_factory(i, path):
            def _on_change(change):
                if change["name"] != "value":
                    return
                if state["updating"]:
                    return

                if change["new"] is True:
                    # enforce single-select
                    state["updating"] = True
                    for j, other in enumerate(selectors):
                        if j != i and other.value:
                            other.value = False
                    # highlight selection
                    for j, c in enumerate(cards):
                        c.layout.border = (
                            "2px solid rgba(255,86,3,0.9)"
                            if j == i
                            else "1px solid rgba(255,86,3,0.25)"
                        )
                    # update preview
                    try:
                        with open(path, "rb") as f:
                            preview.value = f.read()
                    finally:
                        state["updating"] = False
                else:
                    # deselected this one; reset border
                    cards[i].layout.border = "1px solid rgba(255,86,3,0.25)"
                    # clear preview if nothing selected
                    if not any(sel.value for sel in selectors):
                        preview.value = b""

            return _on_change

        cb.observe(on_change_factory(idx, p), names="value")

    # Select the first image by default (triggers preview + highlight)
    if selectors:
        selectors[0].value = True

    grid = widgets.GridBox(
        cards,
        layout=widgets.Layout(
            grid_template_columns="repeat(auto-fit, minmax(160px, 1fr))",
            grid_gap="10px",
        ),
    )

    gallery = widgets.VBox(
        [
            widgets.HTML(
                '<div class="section-title">📥 Click on the check-box for an image in ./medical_images</div>'
            ),
            widgets.HBox(
                [grid, preview],
                layout=widgets.Layout(align_items="flex-start", gap="20px"),
            ),
            widgets.HTML(
                '<span style="color:#F0F0F0;opacity:.8">Only one image can be selected</span>'
            ),
        ],
        layout=widgets.Layout(
            padding="25px",
            background="rgba(0, 35, 60, 0.8)",
            border="2px solid rgba(255, 86, 3, 0.8)",
            border_radius="12px",
            margin="20px 0",
        ),
    )


# Expose a helper to read the current selection
def get_selected_path():
    """Return the single selected file path or None."""
    try:
        selected = next((p for p, cb in zip(image_paths, selectors) if cb.value), None)
        return selected
    except (NameError, IndexError):
        return None


# Question input widget
question_input = widgets.Textarea(
    placeholder="Example: What objects are visible in this image?",
    description="❓ Question:",
    style={"description_width": "130px"},
    layout=widgets.Layout(width="100%", height="90px", margin="5px 0"),
    rows=4,
)

# Final input section
input_section = widgets.VBox(
    [
        widgets.HTML('<div class="section-title">📥 Input Section</div>'),
        gallery,
        question_input,
    ],
    layout=widgets.Layout(
        padding="25px",
        background="rgba(0, 35, 60, 0.8)",
        border="2px solid rgba(255, 86, 3, 0.8)",
        border_radius="12px",
        margin="20px 0",
    ),
)

# Section 2: Action Section
submit_button = widgets.Button(
    description="🚀 Process Image",
    button_style="success",
    tooltip="Click to analyze your image",
    layout=widgets.Layout(width="220px", height="50px"),
    style={"font_weight": "bold"},
)

status_label = widgets.HTML(
    value='<div style="text-align: center; color: #121111; font-size: 15px; padding: 10px; background: rgba(255, 165, 0, 0.1); border-radius: 8px; border: 1px solid rgba(255, 86, 3, 0.8);">⚡ Ready to process</div>',
    layout=widgets.Layout(margin="15px 0 0 0"),
)

action_section = widgets.VBox(
    [
        widgets.HTML('<div class="section-title">⚙️ Action Section</div>'),
        widgets.HBox([submit_button], layout=widgets.Layout(justify_content="center")),
        status_label,
    ],
    layout=widgets.Layout(
        padding="25px",
        background="rgba(0, 35, 60, 0.8)",
        border="2px solid rgba(255, 86, 3, 0.8)",
        border_radius="12px",
        margin="20px 0",
    ),
)

# Section 3: Processing/Loader Section
loader = widgets.HTML(value="", layout=widgets.Layout(margin="0"))

loader_section = widgets.VBox(
    [loader], layout=widgets.Layout(padding="0px", margin="0")
)

# Section 4: Output Section
result_output = widgets.HTML(
    value="""<div style="background: rgba(0, 35, 60, 0.6); 
                        padding: 30px; 
                        border-radius: 10px; 
                        min-height: 180px;
                        border: 2px dashed rgba(255, 86, 3, 0.8);
                        text-align: center;">
                <p style="color: #F0F0F0; font-size: 16px; margin: 60px 0;">
                    ✨ Awaiting results...
                </p>
             </div>""",
    layout=widgets.Layout(width="100%", margin="0"),
)

output_section = widgets.VBox(
    [widgets.HTML('<div class="section-title">📊 Output Section</div>'), result_output],
    layout=widgets.Layout(
        padding="25px",
        background="rgba(0, 35, 60, 0.8)",
        border="2px solid rgba(255, 86, 3, 0.8)",
        border_radius="12px",
        margin="20px 0",
    ),
)


# Function to process the form
def process_form():
    """
    Process the image selected in Section 1 and the question entered there,
    and return HTML formatted results.
    """
    global counter

    # 1) Read UI inputs
    img_path = get_selected_path()
    if not img_path:
        raise ValueError("No image selected. Please pick an image in Section 1.")

    file_name = os.path.basename(img_path)
    question = (question_input.value or "").strip()

    # 2) Read file bytes
    with open(img_path, "rb") as f:
        file_content = f.read()

    # 3) Simulate processing with random delay
    processing_time = random.choice([11, 10, 12, 14])
    time.sleep(processing_time)

    # 4) Get prediction from results
    counter += 1
    result_idx = data_dict.get(counter, 0)

    result_df = visual_question_pdf[visual_question_pdf["id"] == result_idx]
    predicted = (
        result_df["predicted_answer"].iloc[0].strip()
        if not result_df.empty and "predicted_answer" in result_df.columns
        else "(no result)"
    )

    result_html = f"""
    <div style="background: #00233c; 
                padding: 30px; 
                border-radius: 12px; 
                border: 2px solid rgba(255, 140, 0, 0.5);
                box-shadow: 0 5px 15px rgba(0,0,0,0.3);">

        <div style="text-align: center; margin-bottom: 20px;">
            <h3 style="color: #ff8c00; margin: 0; font-size: 24px;">✅ Processing Complete</h3>
        </div>

        <div style="background: rgba(0, 0, 0, 0.3); 
                    padding: 20px; 
                    border-radius: 10px; 
                    margin: 20px 0;
                    border: 1px solid rgba(255, 86, 3, 0.1);">
            <table style="width: 100%; color: #F0F0F0; font-size: 18px; border: 2px;">
                <tr>
                    <td style="padding: 8px; width: 30%;"><strong style="color: #ff8c00;">📷 Image Name:</strong></td>
                    <td style="padding: 8px; color: #F0F0F0;">{file_name}</td>
                </tr>
                <tr>
                    <td style="padding: 8px;"><strong style="color: #ff8c00;">❓ Question:</strong></td>
                    <td style="padding: 8px; color: #F0F0F0;">{question}</td>
                </tr>
            </table>
        </div>

        <div style="background: rgba(255, 140, 0, 0.1); 
                    padding: 20px; 
                    border-radius: 10px; 
                    margin: 20px 0;
                    border-left: 2px solid #ff8c00;">
            <p style="margin: 0 0 10px 0; color: #ff8c00; font-size: 24px; font-weight: bold;">💡 Analysis Result:</p>
            <p style="margin: 0; color: #F0F0F0; line-height: 1.6; font-size: 20px;">
                {predicted}
            </p>
        </div>

        <div style="text-align: center; 
                    margin-top: 20px; 
                    padding-top: 15px; 
                    border-top: 1px solid rgba(255, 86, 3, 0.3);">
            <p style="color: #F0F0F0; font-size: 13px; margin: 0; opacity: 0.8;">
                ⏱️ Processing completed in {processing_time} seconds | ✓ File content extracted successfully
            </p>
        </div>
    </div>
    """
    return result_html


# Submit button click handler
def on_submit_clicked(b):
    """Handle image processing submission."""
    submit_button.disabled = True
    loader_section.layout.display = ""
    status_label.value = (
        '<div style="text-align:center;color:#4e4f46;">⏳ Processing...</div>'
    )

    img_path = get_selected_path()
    file_name = os.path.basename(img_path) if img_path else None

    def worker():
        """Worker thread for non-blocking image processing."""
        try:
            html_out = process_form()
            result_output.value = html_out
            display_name = html_module.escape(file_name or "image")
            status_label.value = f'<div style="text-align:center;color:#81f542">✅ Processed: {display_name}</div>'
        except Exception as e:
            error_msg = html_module.escape(str(e))
            result_output.value = (
                f'<div style="color:#ed0c31;">❌ Error: {error_msg}</div>'
            )
            status_label.value = (
                '<div style="text-align:center;color:#F0F0F0;">❌ Failed</div>'
            )
            print(f"Processing error: {e}")  # Log to console for debugging
        finally:
            loader_section.layout.display = "none"
            submit_button.disabled = False

    # Clear any existing handlers and attach new one
    submit_button._click_handlers.callbacks.clear()
    threading.Thread(target=worker, daemon=True).start()


# Attach event handler
submit_button.on_click(on_submit_clicked)

# Build complete application
app_header = widgets.HTML(
    custom_css
    + """
    <div class="app-title">🎨 Medical Visual Question</div>
    <div class="app-subtitle">Pick a medical image and ask questions to get intelligent insights</div>
    """
)

# Assemble all sections
main_app = widgets.VBox(
    [app_header, input_section, action_section, loader_section, output_section],
    layout=widgets.Layout(
        padding="40px",
        background="#00233c",
        border_radius="20px",
        box_shadow="0 15px 35px rgba(0,0,0,0.4)",
        max_width="900px",
        margin="30px auto",
    ),
)

# Display the application
display(main_app)

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>5.9 Sample questions</b></p>
<p style = 'font-size:16px;font-family:Arial;'>Here are some sample questions you can ask about the images.</p>

<ol style = 'font-size:16px;font-family:Arial;'>
<li>Where are liver stem cells (oval cells) located? </li>
<li>What are stained here with an immunohistochemical stain for cytokeratin 7?</li>
<li>What do the areas of white chalky deposits represent? </li>
<li>Is embolus derived from a lower - extremity deep venous thrombus lodged in a pulmonary artery branch?</li>
<li>How is hyperplasia without atypia characterized? </li>
    </ol>

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>6. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>6.1 Delete your OAF Container</b></p>
<p style="font-size:16px;font-family:Arial">Executing this cell is optional. If you will be executing more OAF use cases, you can leave your OAF environment.</p>

In [ ]:
# Remove your default user environment

try:
    result = remove_env(environment_name)
    print(f"Environment {environment_name} removed!")
except Exception as e:
    print(f"Could not remove the environment, {environment_name}!")
    print("Error:", str(e))

<p style = 'font-size:18px;font-family:Arial'><b>6.2 Remove your database Context</b></p>
<p style="font-size:16px;font-family:Arial">Please remove your context after you've completed this notebook.

In [ ]:
try:
    result = remove_context()
    print("Context removed!")
except Exception as e:
    print("Could not remove the Context!")
    print("Error:", str(e))

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial;'>More information about using this model can be found here: <a href='https://www.kaggle.com/code/basu369victor/blip-medical-visual-question-answering'>Kaggle - Medical Visual Question Answering</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>